Author of code: William Godel

Date: 07/02

Purpose: to test performance of models on alternative data distributions

## Data IN: 

test_data_98.csv

test_data_90.csv

test_data_75.csv

test_data_10.csv


test_data_98_highpol.csv

test_data_90_highpol.csv

test_data_75_highpol.csv

test_data_10_highpol.csv

grid_rf.p

model_nn_highpol.pt

model_large.pt

## Data OUT:

no data out, just statistics


Machine: My laptop or Imac


In [1]:
%load_ext autoreload
%autoreload 

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from collections import Counter
laptop = True
import pickle

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from functions import count_mode, bayes_probs, bayes_binary
from ml_functions import feature_creation

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

#crowd_size, feature_transform

from path import *

In [2]:
test_data_98 = pd.read_csv(prepared_data + "test_data_98.csv")
test_data_90 = pd.read_csv(prepared_data + "test_data_90.csv")
test_data_75 = pd.read_csv(prepared_data + "test_data_75.csv")
test_data_10 = pd.read_csv(prepared_data + "test_data_10.csv")

test_data_98_target = test_data_98['mode'] == 'FM'
test_data_90_target = test_data_90['mode'] == 'FM'
test_data_75_target = test_data_75['mode'] == 'FM'
test_data_10_target = test_data_10['mode'] == 'FM'

In [3]:
X_test_data_98 = feature_creation(test_data_98)
X_test_data_90 = feature_creation(test_data_90)
X_test_data_75 = feature_creation(test_data_75)
X_test_data_10 = feature_creation(test_data_10)

X_test_data_98_large = feature_creation(test_data_98, crowd_num = 25)
X_test_data_90_large = feature_creation(test_data_90, crowd_num = 25)
X_test_data_75_large = feature_creation(test_data_75, crowd_num = 25)
X_test_data_10_large = feature_creation(test_data_10, crowd_num = 25)

In [4]:
numeric_features = []

resp_var = [x for x in X_test_data_98.columns if 'resp_veracity_' in x]
new_cols = [x for x in X_test_data_98.columns if 'new' in x]

#numeric_features.extend(resp_var)
numeric_features.extend(resp_var) #rempve this if it doesn't work
numeric_features.extend(new_cols)
numeric_features.extend(["crowd_means",'crowd_median','crowd_full_range', 'crowd_IQR_range', \
                         'crowd_variance', 'crowd_bayes'])



In [5]:
def conf_eval(preds, truth):
    
    if True in np.unique(truth):
        
        condition_pos = True
        condition_neg = False
        
    else:
        
        condition_pos = 1
        condition_neg = 0
        
    con_pos_count = np.sum(truth == condition_pos)
    con_neg_count = np.sum(truth == condition_neg)
        
    accuracy = np.sum((preds == truth))/truth.size
    
    True_pos = np.sum(preds[truth == condition_pos] == truth[truth ==condition_pos])
    False_pos = np.sum(preds[truth == condition_pos] != truth[truth ==condition_pos])
    
    True_neg = np.sum(preds[truth == condition_neg] == truth[truth == condition_neg])
    False_neg = np.sum(preds[truth == condition_neg] != truth[truth == condition_neg])
    
    
    return accuracy, True_pos, False_pos, True_neg, False_neg
    
    
def conf_perc(a_list):
    
    total_pos = sum(a_list[2:4])
    total_neg = sum(a_list[4:])
    print("TP", "FP", "TN", "FN")
    return a_list[2]/total_pos, a_list[3]/total_pos, a_list[4]/total_neg, a_list[5]/total_neg

    

### Random Forest
### standard crowd
### size 10

In [6]:
performance_dic_98 = {}
performance_dic_90 = {}
performance_dic_75 = {}
performance_dic_10 = {}

In [7]:
grid_rf = pickle.load(open(models + 'grid_rf.p', "rb" ))

algo = grid_rf

algo_name = "randomforest_10"

all_results = [algo_name]
results = list(conf_eval(algo.predict(X_test_data_98), test_data_98_target))
all_results.extend(results)
performance_dic_98[algo_name] = all_results

all_results = [algo_name]
results = list(conf_eval(algo.predict(X_test_data_90), test_data_90_target))
all_results.extend(results)
performance_dic_90[algo_name] = all_results

all_results = [algo_name]
results = list(conf_eval(algo.predict(X_test_data_75), test_data_75_target))
all_results.extend(results)
performance_dic_75[algo_name] = all_results

results = [algo_name]
all_results = list(conf_eval(algo.predict(X_test_data_10), test_data_10_target))
all_results.extend(results)
performance_dic_10[algo_name] = all_results


In [8]:
conf_perc(performance_dic_98['randomforest_10'])

TP FP TN FN


(0.48947368421052634,
 0.5105263157894737,
 0.6969924812030075,
 0.3030075187969925)

In [9]:
performance_dic_98['randomforest_10']

['randomforest_10', 0.6928421052631579, 93, 97, 6489, 2821]

## Qualified Crowds

### loading data

In [10]:
test_data_98_highpk = pd.read_csv(prepared_data + 'test_data_98_highpol.csv')
test_data_90_highpk = pd.read_csv(prepared_data + "test_data_90_highpol.csv")
test_data_75_highpk = pd.read_csv(prepared_data + "test_data_75_highpol.csv")
test_data_10_highpk = pd.read_csv(prepared_data + "test_data_10_highpol.csv")


X_test_data_98_highpk = feature_creation(test_data_98_highpk)
X_test_data_90_highpk = feature_creation(test_data_90_highpk)
X_test_data_75_highpk = feature_creation(test_data_75_highpk)
X_test_data_10_highpk = feature_creation(test_data_10_highpk)

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self, emb_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(emb_dim, 500) 
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 25)
        self.fc4 = nn.Linear(25, 25)
        self.fc5 = nn.Linear(25, 10)
        self.fc6 = nn.Linear(10, 1)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        
        return x
    
    
from torch.utils.data import Dataset

class data_loader(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, data_list, target_list):
        """
        @param data_list: list of newsgroup tokens 
        @param target_list: list of newsgroup targets 

        """
        self.data_list = data_list.values
        self.target_list = target_list
        assert (len(self.data_list) == len(self.target_list))

    def __len__(self):
        return len(self.data_list)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        
        data = self.data_list[key]
        label = self.target_list[key]
        return [data, label]


In [12]:

    
emb_dim = 61   
net_large = Net(emb_dim)
net_large.load_state_dict(torch.load(models + "model_nn_highpol.pt"))
net_large.eval()

# Preparing the data
y_data_bin_large = np.where(test_data_98_target == True, 1, 0)
data = X_test_data_98_highpk
nn_list_large = ["nn_10_pk"]
test = StandardScaler()
test.fit(data[numeric_features])
#training data
X_train_stand_large = data.copy()
X_train_stand_large[numeric_features] = test.transform(data[numeric_features])
train_dat_nn_large = data_loader(X_train_stand_large,y_data_bin_large)
nn_preds  = torch.sigmoid(net_large(torch.tensor(train_dat_nn_large[:][0]).float()))
nn_preds = np.where(nn_preds > .5, 1,0).reshape(-1,)
all_results = list(conf_eval(nn_preds, y_data_bin_large))

nn_list_large.extend(all_results)
performance_dic_98['nn_10_pk'] = nn_list_large



# Preparing the data
y_data_bin_large = np.where(test_data_90_target == True, 1, 0)
data = X_test_data_90_highpk
nn_list_large = ["nn_25"]
test = StandardScaler()
test.fit(data[numeric_features])
#training data
X_train_stand_large = data.copy()
X_train_stand_large[numeric_features] = test.transform(data[numeric_features])
train_dat_nn_large = data_loader(X_train_stand_large,y_data_bin_large)
nn_preds  = torch.sigmoid(net_large(torch.tensor(train_dat_nn_large[:][0]).float()))
nn_preds = np.where(nn_preds > .5, 1,0).reshape(-1,)
all_results = list(conf_eval(nn_preds, y_data_bin_large))

nn_list_large.extend(all_results)
performance_dic_90['nn_10_pk'] = nn_list_large


# Preparing the data
y_data_bin_large = np.where(test_data_75_target == True, 1, 0)
data = X_test_data_75_highpk
nn_list_large = ["nn_10_pk"]
test = StandardScaler()
test.fit(data[numeric_features])
#training data
X_train_stand_large = data.copy()
X_train_stand_large[numeric_features] = test.transform(data[numeric_features])
train_dat_nn_large = data_loader(X_train_stand_large,y_data_bin_large)
nn_preds  = torch.sigmoid(net_large(torch.tensor(train_dat_nn_large[:][0]).float()))
nn_preds = np.where(nn_preds > .5, 1,0).reshape(-1,)
all_results = list(conf_eval(nn_preds, y_data_bin_large))

nn_list_large.extend(all_results)
performance_dic_75['nn_10_pk'] = nn_list_large




# Preparing the data
y_data_bin_large = np.where(test_data_10_target == True, 1, 0)
data = X_test_data_10_highpk
nn_list_large = ["nn_10_pk"]
test = StandardScaler()
test.fit(data[numeric_features])
#training data
X_train_stand_large = data.copy()
X_train_stand_large[numeric_features] = test.transform(data[numeric_features])
train_dat_nn_large = data_loader(X_train_stand_large,y_data_bin_large)
nn_preds  = torch.sigmoid(net_large(torch.tensor(train_dat_nn_large[:][0]).float()))
nn_preds = np.where(nn_preds > .5, 1,0).reshape(-1,)
all_results = list(conf_eval(nn_preds, y_data_bin_large))

nn_list_large.extend(all_results)
performance_dic_10['nn_10_pk'] = nn_list_large



In [13]:
performance_dic_10['nn_10_pk']

['nn_10_pk', 0.5401052631578948, 4347, 4203, 784, 166]

In [14]:
performance_dic_90

{'randomforest_10': ['randomforest_10',
  0.6980734814190968,
  442,
  507,
  6189,
  2361],
 'nn_10_pk': ['nn_25', 0.7070217917675545, 587, 362, 6129, 2421]}

### Neural Network
### Large crowd
### size 25


In [15]:
numeric_features_large = []

resp_var = [x for x in X_test_data_98_large.columns if 'resp_veracity_' in x]
new_cols = [x for x in X_test_data_98_large.columns if 'new' in x]

#numeric_features.extend(resp_var)
numeric_features_large.extend(resp_var) #rempve this if it doesn't work
numeric_features_large.extend(new_cols)
numeric_features_large.extend(["crowd_means",'crowd_median','crowd_full_range', 'crowd_IQR_range', \
                         'crowd_variance', 'crowd_bayes'])

In [16]:

emb_dim = 121   
net_large = Net(emb_dim)
net_large.load_state_dict(torch.load(models + "model_large.pt"))
net_large.eval()



# Preparing the data
y_data_bin_large = np.where(test_data_98_target == True, 1, 0)
data = X_test_data_98_large
nn_list_large = ["nn_25"]
test = StandardScaler()
test.fit(data[numeric_features_large])
#training data
X_train_stand_large = data.copy()
X_train_stand_large[numeric_features_large] = test.transform(data[numeric_features_large])
train_dat_nn_large = data_loader(X_train_stand_large,y_data_bin_large)
nn_preds  = torch.sigmoid(net_large(torch.tensor(train_dat_nn_large[:][0]).float()))
nn_preds = np.where(nn_preds > .5, 1,0).reshape(-1,)
all_results = list(conf_eval(nn_preds, y_data_bin_large))

nn_list_large.extend(all_results)
performance_dic_98['nn_25'] = nn_list_large



# Preparing the data
y_data_bin_large = np.where(test_data_90_target == True, 1, 0)
data = X_test_data_90_large
nn_list_large = ["nn_25"]
test = StandardScaler()
test.fit(data[numeric_features_large])
#training data
X_train_stand_large = data.copy()
X_train_stand_large[numeric_features_large] = test.transform(data[numeric_features_large])
train_dat_nn_large = data_loader(X_train_stand_large,y_data_bin_large)
nn_preds  = torch.sigmoid(net_large(torch.tensor(train_dat_nn_large[:][0]).float()))
nn_preds = np.where(nn_preds > .5, 1,0).reshape(-1,)
all_results = list(conf_eval(nn_preds, y_data_bin_large))

nn_list_large.extend(all_results)
performance_dic_90['nn_25'] = nn_list_large



# Preparing the data
y_data_bin_large = np.where(test_data_75_target == True, 1, 0)
data = X_test_data_75_large
nn_list_large = ["nn_25"]
test = StandardScaler()
test.fit(data[numeric_features_large])
#training data
X_train_stand_large = data.copy()
X_train_stand_large[numeric_features_large] = test.transform(data[numeric_features_large])
train_dat_nn_large = data_loader(X_train_stand_large,y_data_bin_large)
nn_preds  = torch.sigmoid(net_large(torch.tensor(train_dat_nn_large[:][0]).float()))
nn_preds = np.where(nn_preds > .5, 1,0).reshape(-1,)
all_results = list(conf_eval(nn_preds, y_data_bin_large))

nn_list_large.extend(all_results)
performance_dic_75['nn_25'] = nn_list_large




# Preparing the data
y_data_bin_large = np.where(test_data_10_target == True, 1, 0)
data = X_test_data_10_large
nn_list_large = ["nn_25"]
test = StandardScaler()
test.fit(data[numeric_features_large])
#training data
X_train_stand_large = data.copy()
X_train_stand_large[numeric_features_large] = test.transform(data[numeric_features_large])
train_dat_nn_large = data_loader(X_train_stand_large,y_data_bin_large)
nn_preds  = torch.sigmoid(net_large(torch.tensor(train_dat_nn_large[:][0]).float()))
nn_preds = np.where(nn_preds > .5, 1,0).reshape(-1,)
all_results = list(conf_eval(nn_preds, y_data_bin_large))

nn_list_large.extend(all_results)
performance_dic_10['nn_25'] = nn_list_large



In [17]:
print(performance_dic_10['nn_25'])
conf_perc(performance_dic_10['nn_25'])

['nn_25', 0.49757894736842107, 3908, 4642, 819, 131]
TP FP TN FN


(0.45707602339181286,
 0.5429239766081871,
 0.8621052631578947,
 0.13789473684210526)

In [18]:
print(performance_dic_75['nn_25'])
conf_perc(performance_dic_75['nn_25'])

['nn_25', 0.7170526315789474, 1294, 1081, 5518, 1607]
TP FP TN FN


(0.5448421052631579,
 0.4551578947368421,
 0.7744561403508772,
 0.2255438596491228)

In [19]:
print(performance_dic_90['nn_25'])
conf_perc(performance_dic_90['nn_25'])

['nn_25', 0.7379724181492788, 553, 396, 6457, 2093]
TP FP TN FN


(0.5827186512118019,
 0.4172813487881981,
 0.7552046783625731,
 0.2447953216374269)

In [20]:
print(performance_dic_98['nn_25'])
conf_perc(performance_dic_98['nn_25'])

['nn_25', 0.738421052631579, 109, 81, 6906, 2404]
TP FP TN FN


(0.5736842105263158,
 0.4263157894736842,
 0.7417830290010741,
 0.2582169709989259)

In [21]:
print(performance_dic_10['nn_10_pk'])
conf_perc(performance_dic_10['nn_10_pk'])

['nn_10_pk', 0.5401052631578948, 4347, 4203, 784, 166]
TP FP TN FN


(0.508421052631579,
 0.49157894736842106,
 0.8252631578947368,
 0.17473684210526316)

In [22]:
print(performance_dic_75['nn_10_pk'])
conf_perc(performance_dic_75['nn_10_pk'])

['nn_10_pk', 0.7055789473684211, 1430, 945, 5273, 1852]
TP FP TN FN


(0.6021052631578947,
 0.39789473684210525,
 0.7400701754385965,
 0.2599298245614035)

In [23]:
print(performance_dic_90['nn_10_pk'])
conf_perc(performance_dic_90['nn_10_pk'])

['nn_25', 0.7070217917675545, 587, 362, 6129, 2421]
TP FP TN FN


(0.6185458377239199, 0.3814541622760801, 0.716842105263158, 0.2831578947368421)

In [24]:
print(performance_dic_98['nn_10_pk'])
conf_perc(performance_dic_98['nn_10_pk'])

['nn_10_pk', 0.7001052631578948, 116, 74, 6535, 2775]
TP FP TN FN


(0.6105263157894737,
 0.3894736842105263,
 0.7019334049409237,
 0.2980665950590763)